# Running back prediction model

## I will ...

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
year_rb_stats = pd.read_csv("data/yearly_player_stats_offense.csv")
week_rb_stats = pd.read_csv("data/weekly_player_stats_offense.csv")
fantasy = pd.read_csv("data/fantasy_data.csv")

In [3]:
# print(year_rb_stats.head())
# print(week_rb_stats.head())
# print(fantasy.head())